In [1]:
import tensorflow as tf

<br>
<br>
<br>
<br>

โจทย์คือเรามีfeature 3 ตัว เป็นเลขเรียงกัน labelคือเลขตัวถัดไป <br>
<table>
<tr><td>x1</td> <td>x2</td>  <td>x3</td>   <td>y</td> </tr>
<tr><td>1</td> <td>2</td>  <td>3</td>   <td>4</td> </tr>
<tr><td>2</td> <td>3</td>  <td>4</td>   <td>5</td> </tr>
<tr><td>..</td> <td>..</td>  <td>..</td>   <td>..</td> </tr>    
</table>

เราอยากจะสร้างโมเดลNNแจ่มๆซักตัว ที่ใส่3ค่าแล้วได้ค่าถัดไปออกมา <br>
เช่นใส่ 10 11 12 ได้ 13 ออกมา

ก่อนอื่นเรามาสร้างtraining setกันก่อนเลย ชิวมาก มีแค่5sample

In [2]:
X = tf.constant([[1.0,2.0,3.0],[2.0,3.0,4.0],[3.0,4.0,5.0],[4.0,5.0,6.0],[5.0,6.0,7.0]])
y = tf.constant([4.0,5.0,6.0,7.0,8.0])

In [3]:
print(X[0])
print(y[0])

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


<br>
<br>
<br>
<br>

สร้างโมเดลนู้บๆมีneuronตัวเดียว (มีแค่ 3 weight 1 bias)

In [4]:
class SingleNeuron(tf.keras.Model):
    def __init__(self):
        super().__init__()                     # inherit สิ่งตั่งต่างมาจากtf.keras.Model
        self.layer1 = tf.keras.layers.Dense(1) # สร้างเลเยอร์ขึ้นมาอันนึง
    
    def call(self, x):                         # method ตัวนี้เอาไว้กำหนดการไหลของข้อมูลว่ามันต้องผ่านอะไรบ้าง
        output = self.layer1(x)                # เราเอาข้อมูลไปผ่านlayer1 ได้ค่าๆนึงออกมา ใส่ไว้ในตัวแปรoutput
        return output

In [5]:
mse = tf.keras.losses.MeanSquaredError()       # loss functionยังไงล่ะเด้อ เอาไว้วัดว่าค่าจริงกับค่าที่โมเดลมันpredictออกมามันต่างกันขนาดไหน

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1) # อันนี้เป็นตัวที่จะเปลี่ยนค่าของweightให้เรา

<br>
<br>
<br>
<br>

In [7]:
model = SingleNeuron() #สร้างโมเดลจากคลาสที่เราเขียนไว้

ลองเอาค่าXยัดใส่โมเดลดู จะรู้ว่าตอนนี้มันยังโง่เหมือนวิน

In [8]:
model(X)

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[3.0236084],
       [3.6894963],
       [4.355384 ],
       [5.0212727],
       [5.68716  ]], dtype=float32)>

<i>จริงๆมันควรจะได้ 4 5 6 7 8</i>

<br>
<br>
<br>

เราเลยลองoptimizeโมเดลดู

In [9]:
with tf.GradientTape() as tape: # tapeมันจะคอยจำให้เราว่าเกิดการคำนวณอะไรขึ้นบ้าง
    y_pred = model(X)
    loss = mse(y, y_pred)
gradient = tape.gradient(loss, model.trainable_variables)           #Backpropagation(ดิฟlossเทียบกับweightแต่ละตัว)
optimizer.apply_gradients(zip(gradient, model.trainable_variables)) #Optimize parameter(ยิ่งgradientมากยิ่งเปลี่ยนweightให้เยอะ)

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [10]:
model(X) #ยังคงโง่

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[3.723608 ],
       [4.689496 ],
       [5.6553836],
       [6.6212716],
       [7.5871596]], dtype=float32)>

<br>
<br>
<br>
<br>

ขี้เกียจเขียนมือเลยวนลูปเอา

In [11]:
for i in range(200):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = mse(y, y_pred)
    gradient = tape.gradient(loss, model.trainable_variables) 
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))
    if i % 10 == 0:
        print(f'Epoch {i+10} : loss = {loss}')

Epoch 10 : loss = 0.12108761072158813
Epoch 20 : loss = 0.22126635909080505
Epoch 30 : loss = 0.14142709970474243
Epoch 40 : loss = 0.0047951340675354
Epoch 50 : loss = 0.0234390702098608
Epoch 60 : loss = 0.008184382691979408
Epoch 70 : loss = 0.001567853381857276
Epoch 80 : loss = 0.0003311033360660076
Epoch 90 : loss = 0.0001335216948064044
Epoch 100 : loss = 8.848532161209732e-05
Epoch 110 : loss = 5.4272055422188714e-05
Epoch 120 : loss = 1.9154345864080824e-05
Epoch 130 : loss = 1.3996809684613254e-06
Epoch 140 : loss = 7.086175628501223e-07
Epoch 150 : loss = 9.549870583214215e-07
Epoch 160 : loss = 1.5899150440645826e-08
Epoch 170 : loss = 1.0874509825953282e-07
Epoch 180 : loss = 5.8062594732177786e-09
Epoch 190 : loss = 1.4438103157488058e-08
Epoch 200 : loss = 8.185452315956354e-12


In [12]:
model(X).numpy()

array([[4.000028 ],
       [5.000034 ],
       [6.0000405],
       [7.000047 ],
       [8.000053 ]], dtype=float32)

โอเคเก่งมาก !!!!

ไหนลองเอาไปใช้กับข้อมูลอื่นดู

In [13]:
unseen_data = tf.constant([[10.0,11.0,12.0]])

model(unseen_data).numpy()

array([[13.000087]], dtype=float32)

เวรี่กูด